In [1]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

In [ ]:
dataset = pd.read_csv("BBC News Train.csv")
print(dataset.head())

In [ ]:
dataset.shape

In [ ]:
dataset['Category'].value_counts()

In [ ]:
# Convert the given news categories into categorical values.
target_category = dataset['Category'].unique()
print(target_category)

In [ ]:
dataset['Category'].factorize()

In [ ]:
# Associate Category names with numerical index and save it in new column CategoryId
dataset['CategoryId'] = dataset['Category'].factorize()[0]
dataset.head()

In [ ]:
# Create a new pandas dataframe "category", which only has unique Categories, also sorting this list in order of CategoryId values
category = dataset[['Category', 'CategoryId']].drop_duplicates().sort_values('CategoryId')
category

In [ ]:
dataset.Category.value_counts().plot(kind = "bar", color = ["purple", "orange", "red", "green", "blue"])
plt.xlabel("Category of data")
plt.title("Articles Assigned to Categories")
plt.show()

In [ ]:
fig = plt.figure(figsize = (5,5))
colors = ["purple", "blue", "red", "green", "orange"]
business = dataset[dataset['CategoryId'] == 0 ]
tech = dataset[dataset['CategoryId'] == 1 ]
politics = dataset[dataset['CategoryId'] == 2]
sport = dataset[dataset['CategoryId'] == 3]
entertainment = dataset[dataset['CategoryId'] == 4]
count = [business['CategoryId'].count(), tech['CategoryId'].count(), politics['CategoryId'].count(), sport['CategoryId'].count(), entertainment['CategoryId'].count()]
pie = plt.pie(count, labels = ['business', 'tech', 'politics', 'sport', 'entertainment'],
              autopct = "%1.1f%%",
              shadow = True,
              colors = colors,
              startangle = 90,
              explode = (0.05, 0.05, 0.05, 0.05,0.05))


In [ ]:
from wordcloud import WordCloud

stop = set(stopwords.words('english'))

business = dataset[dataset['CategoryId'] == 0]
business = business['Text']

tech = dataset[dataset['CategoryId'] == 1]
tech = tech['Text']

politics = dataset[dataset['CategoryId'] == 2]
politics = politics['Text']

sport = dataset[dataset['CategoryId'] == 3]
sport = sport['Text']

entertainment = dataset[dataset['CategoryId'] == 4]
entertainment = entertainment['Text']

def wordcloud_draw(dataset, color = 'white'):

    words = ' '.join(dataset)

    cleaned_word = ' '.join([word for word in words.split() if (word != 'news' and word != 'text')])

    wordcloud = WordCloud(stopwords = stop, 
                          background_color = color,
                          width = 400, 
                          height = 400).generate(cleaned_word)

    plt.figure(1, figsize = (10,7))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

print("business related words:")
wordcloud_draw(business, 'white')

print("tech related words:")
wordcloud_draw(tech, 'white')

print("politics related words:")
wordcloud_draw(politics, 'white')

print("sport related words:")
wordcloud_draw(sport, 'white')

print("entertainment related words:")
wordcloud_draw(entertainment, 'white')


In [ ]:
# remove all tags in dataset
def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
dataset['Text'] = dataset['Text'].apply(remove_tags)

In [ ]:
# remove all special characters in dataset
def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
dataset['Text'] = dataset['Text'].apply(special_char)

In [ ]:
def convert_lower(text):
   return text.lower()
dataset['Text'] = dataset['Text'].apply(convert_lower)
dataset['Text'][1]

In [ ]:
def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return [x for x in words if x not in stop_words]
dataset['Text'] = dataset['Text'].apply(remove_stopwords)
dataset['Text'][1]

In [ ]:
nltk.download('omw-1.4')

In [ ]:
def lemmatize_word(text):
  wordnet = WordNetLemmatizer()
  return " ".join([wordnet.lemmatize(word) for word in text])
dataset['Text'] = dataset['Text'].apply(lemmatize_word)
dataset['Text'][1]

In [ ]:
dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
x = np.array(dataset.iloc[:,0].values)
y = np.array(dataset.CategoryId.values)
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(dataset.Text).toarray()
print("X.shape = ",x.shape)
print("y.shape = ",y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0, shuffle = True)
print(len(x_train))
print(len(x_test))

In [ ]:
#create list of model and accuracy dicts
perform_list = [ ]

In [ ]:
def run_model(model_name, est_c, est_pnlty):

    mdl=''

    if model_name == 'Logistic Regression':
        mdl = LogisticRegression()
    elif model_name == 'Random Forest':
        mdl = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
    elif model_name == 'Multinomial Naive Bayes':
        mdl = MultinomialNB(alpha=1.0,fit_prior=True)
    elif model_name == 'Support Vector Classifier':
        mdl = SVC()
    elif model_name == 'Decision Tree Classifier':
        mdl = DecisionTreeClassifier()
    elif model_name == 'K Nearest Neighbor k=10':
        mdl = KNeighborsClassifier(n_neighbors=10 , metric= 'minkowski' , p = 4)
    elif model_name == 'K Nearest Neighbor k=7':
        mdl = KNeighborsClassifier(n_neighbors=7 , metric= 'minkowski' , p = 4)
    elif model_name == 'Gaussian Naive Bayes':
        mdl = GaussianNB()
    elif model_name == 'Multilayer Perceptron Classifier':
        mdl = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(6,), random_state=1)
        

    oneVsRest = OneVsRestClassifier(mdl)

    oneVsRest.fit(x_train, y_train)

    y_pred = oneVsRest.predict(x_test)

    # Performance metrics
    accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

    # Get precision, recall, f1 scores
    precision, recall, f1score, support = score(y_test, y_pred, average='micro')

    print(f'Test Accuracy Score of Basic {model_name}: % {accuracy}')
    print(f'Precision : {precision}')
    print(f'Recall : {recall}')
    print(f'F1-score : {f1score}')

    # Add performance parameters to list
    perform_list.append(dict([
        ('Model', model_name),
        ('Test Accuracy', round(accuracy, 2)),
        ('Precision', round(precision, 2)),
        ('Recall', round(recall, 2)),
        ('F1', round(f1score, 2))
        ])
    )

In [ ]:
run_model('Logistic Regression', est_c=None, est_pnlty=None)


In [ ]:
run_model('Random Forest', est_c=None, est_pnlty=None)

In [ ]:
run_model('Multinomial Naive Bayes', est_c=None, est_pnlty=None)

In [ ]:
run_model('Support Vector Classifier', est_c=None, est_pnlty=None)

In [ ]:
run_model('Decision Tree Classifier', est_c=None, est_pnlty=None)

In [ ]:
run_model('Gaussian Naive Bayes', est_c=None, est_pnlty=None)

In [ ]:
run_model('K Nearest Neighbor k=10', est_c=None, est_pnlty=None)

In [ ]:
run_model('K Nearest Neighbor k=7', est_c=None, est_pnlty=None)

In [ ]:
# This a neural network
run_model('Multilayer Perceptron Classifier', est_c=None, est_pnlty=None)

In [ ]:
model_performance = pd.DataFrame(data=perform_list)
model_performance = model_performance[['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1']]
model_performance

In [ ]:
# classifier = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0).fit(x_train, y_train)
# classifier
# y_pred = classifier.predict(x_test)

mdl0 = LogisticRegression()

mdl1 = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)

mdl2 = MultinomialNB(alpha=1.0,fit_prior=True)

mdl3 = SVC()

mdl4 = DecisionTreeClassifier()

mdl5 = GaussianNB()

mdl6 = KNeighborsClassifier(n_neighbors=10 , metric= 'minkowski' , p = 4)

mdl7 = KNeighborsClassifier(n_neighbors=7 , metric= 'minkowski' , p = 4)

mdl8 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(6,), random_state=1)

classifier = OneVsRestClassifier(mdl5)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [ ]:
y_pred1 = cv.transform(['I took a film class for actors']).toarray()
yy = classifier.predict(y_pred1)
result = ""
if yy == [0]:
  result = "Business News"
elif yy == [1]:
  result = "Tech News"
elif yy == [2]:
  result = "Politics News"
elif yy == [3]:
  result = "Sports News"
elif yy == [4]:
  result = "Entertainment News"
print(result)

In [ ]:
# I set the default classifier to the most accurate classifier

def phrase_category(phrase, model = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)):
    classifier = OneVsRestClassifier(model)
    classifier.fit(x_train, y_train)
    
    y_pred = cv.transform([phrase]).toarray()
    yy = classifier.predict(y_pred)
    
    result = ""
    if yy == [0]:
      result = "Business News"
    elif yy == [1]:
      result = "Tech News"
    elif yy == [2]:
      result = "Politics News"
    elif yy == [3]:
      result = "Sports News"
    elif yy == [4]:
      result = "Entertainment News"
    
    print(f"\nThe model: {model}")
    print(f"The phrase: '{phrase[:30]}...'")
    print(f"The category: {result}")

In [ ]:
model_list = [mdl0, mdl1, mdl2, mdl3, mdl4, mdl5, mdl6, mdl7, mdl8]

In [ ]:
# actual category: sports
string = "I want to play kickball"

phrase_category(string)

for mdl_i in model_list:
    phrase_category(string, mdl_i)

In [ ]:
# actual category: for fun
sentence = "high schoolers are always depressed"
for mdl_i in model_list:
    phrase_category(sentence, mdl_i)

In [ ]:
# actual category: tech news
sentence = "Subject: Re: Electronic Tesla Coils First of all, realize that Tesla invented AC power generators, motors, transformers, conductors, etc. Technically, *ALL* transformers are Tesla coils.  In general though when someone refers to a Tesla coil, they mean an 'air core resonant transformer'. The TV flyback version Tesla coil (see the _Encyclopedia_of_Electronic_Circuits_ V3, 106-1 for diagram) has NOT an air core. It is of a class of circuit called 'Oscillating Shuttle Circuit' (OSC). Generally OSC's are highly efficient, but this version uses transistors and resistors, which are very lossy devices. Typically Tesla used active reactances instead of passive resistors, so that he could achieve efficiencies of 99.5%, and better. The usual application of an air-core resonant transformer, or of an OSC, is to produce strong EMI for wireless broadcasts. How well do you think your computer screen would work if we removed the HF HV Tesla (flyback) coil from it? If we were to remove from our homes and industries all Tesla coils, our lights would go dark, our cars would sputter and die, our radios would go silent, our industries would grind to a halt, and we would have to go back to using coal for heat, gas for lamps, horses for transportation, steam for power, and telegraph for communication. Is that real world enough for you??????? GET THE MESSAGE! WE WOULD NOT HAVE 1/100 THE CONVIENIENCES WE HAVE TODAY IF NOT FOR TESLA. GIVE CREDIT WHERE CREDIT IS DUE! If it had been up to Edison, we'd still be in the 19th century. (flame me at your own peril. I'm very good at putting edison down).  'Tesla was 100 years ahead of his time. Perhaps now his time comes'."
for mdl_i in model_list:
    phrase_category(sentence, mdl_i)

In [ ]:
# actual category: political paper 
# directory :useless files/20-newsgroups/talk.politics.misc/176845
sentence = "Furthermore, what are the specific charges against the four LAPD officers? Which civil rights or laws are they accused of violating? I believe it is a general charge, that is no specific right is mentioned. I don't think that this is accurate. I believe, and could be wrong, that there IS a specific right allegedly to have been violated, like the 14th or due process or whatever. What about double jeopardy? Has there been any concern that a verdict against Koon, et al. might be overturned upon appeal because they're being tried again for the same actions? (I thought I heard something on the news about this.) The SS has previously ruled that since the seperate governments were in essence seperate sovereigns, then double jeopardy does not apply. (If this is true, then could defendents also be tried under city and county governments?) This mornings paper said that the ACLU has decided to reinstate itsopposition to this kind of thing. They had earlier suspended their opposition while they examined the King case. There might be hope for the ACLU after all. Double jeopardy does not apply, but not for the reasons you quote. Double jeopardy states that a person may not be tried twice on the same charge. However, the police are not on trial for the crime of excessive force or assault. They are NOW on trial for the DIFFERENT crime of violating Mr. King's civil rights. AS for the city and county or state trying you more than once, it most likely will not happen. This is because cities and states have separate laws governing behaviour. For example, in some states, it is an offence to carry marijuana, but not a city offence. Also, I think murder is against federal, but not some state laws."
for mdl_i in model_list:
    phrase_category(sentence, mdl_i)

In [ ]:
# most accurate at classifications in outside tests are:
    # [Index | Classification]
    # [1     | RandomForestClassifier]
    # [2     | MultinomialNB]
    # [7     | MLPClassifier]